In [1]:
import pathlib
import threading
import os
import shutil
import time
import concurrent.futures
import pandas as pd
import multiprocessing
import json
import pandas as pd


In [2]:
path_files = ['Processing', 'Queue', 'Processed']

In [3]:
for i in path_files:
  if not pathlib.Path(i).exists():
    pathlib.Path(i).mkdir(exist_ok=True)
!ls

Processed  Processing  Queue  sample_data


In [4]:
def check_empty():
  if len(os.listdir(pathlib.Path(path_files[1]))) == 0:
    return True
  return False



In [ ]:
a= 1
b = 101

In [5]:
def add_files_to_Processing():
  for file_number in range(a,b):
    folder_name = pathlib.Path(path_files[0])
    file_name = f'file{file_number}.txt'
    new_file = folder_name/file_name
    
    #print('sleep for 1 sec')
    open(new_file, 'a').close()
    print(f'added {file_name} to {path_files[0]} folder')
    time.sleep(1)
    #print('Done for 1 sec')
  
  


In [6]:
def move_file_from_processing2queue():
  for file_number in range(a,b):
    prev_location = pathlib.Path(f'{path_files[0]}/file{file_number}.txt')
    new_location = pathlib.Path(f'{path_files[1]}/file{file_number}.txt')
    while not check_empty:
      time.sleep(5)
    
   
  
    pathlib.Path(prev_location).rename(pathlib.Path(new_location))
    print('moved from processing to queue')
    #print('sleeping for 5 seconds')
    time.sleep(5)
    #print('Done for 5 sec')
     
 
  


In [7]:
file_names_list = []
file_number_list = []


In [9]:

def move_file_from_queue2processed():
  for file_number in range(a,b):
    prev_location = pathlib.Path(f'{path_files[1]}/file{file_number}.txt')
    new_location = pathlib.Path(f'{path_files[2]}/file{file_number}.txt')
    
    pathlib.Path(prev_location).rename(pathlib.Path(new_location))
    print('moved from queue to processed')
    
    file_number_list.append(file_number)
    print(file_number_list)
    file_names_list.append(f'file{file_number}.txt')
    print(file_names_list)
    #print('sleeping for 10 secs')
    time.sleep(10)
    
    #print('Done sleeping for 10 sec')
  x = dict(zip(file_number_list, file_names_list))
  y = pd.DataFrame.from_dict(data=x, orient='index', columns=['file_names_list'])
  y.to_csv('sample.csv')
  
  #with open("sample.json", "w") as outfile:
   # json.dump(x, outfile)
  
    

  
  



In [10]:

f1 = multiprocessing.Process(target=add_files_to_Processing)
f2 = multiprocessing.Process(target=move_file_from_processing2queue)
f3 = multiprocessing.Process(target=move_file_from_queue2processed)

f1.start()
f2.start()
f3.start()

f1.join()
f2.join()
f3.join()
  
  




moved from processing to queue
added file1.txt to Processing folder
moved from queue to processed
[1]
['file1.txt']
added file2.txt to Processing folder
added file3.txt to Processing folder
added file4.txt to Processing folder
moved from processing to queue
moved from processing to queue
moved from queue to processed
[1, 2]
['file1.txt', 'file2.txt']
moved from processing to queue
moved from queue to processed
[1, 2, 3]
['file1.txt', 'file2.txt', 'file3.txt']
moved from queue to processed
[1, 2, 3, 4]
['file1.txt', 'file2.txt', 'file3.txt', 'file4.txt']


In [11]:
df = pd.read_csv('sample.csv', index_col=None)
df

,Unnamed: 0,file_names_list
0,1,file1.txt
1,2,file2.txt
2,3,file3.txt
3,4,file4.txt


In [12]:
df=df.rename(columns = {'Unnamed: 0':'file_number_list'})

In [13]:
df

,file_number_list,file_names_list
0,1,file1.txt
1,2,file2.txt
2,3,file3.txt
3,4,file4.txt


In [14]:
import sqlite3

conn = sqlite3.connect('test.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS file_data(file_number_list integer, 
                      file_names_list text);''') 
                     
#df = pd.read_csv('sample.csv', index_col=None)
df.to_sql('file_data', conn, if_exists='append', index=False)
conn.commit()

print("Table created successfully");

conn.close()




Opened database successfully
Table created successfully


In [15]:
conn = sqlite3.connect('test.db')
cur = conn.cursor()
cur.execute("SELECT * FROM file_data")
rows = cur.fetchall()
for row in rows:
  print(row)


(1, 'file1.txt')
(2, 'file2.txt')
(3, 'file3.txt')
(4, 'file4.txt')
